In [1]:
from IPython import get_ipython
import numpy as np
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, minmax_scale
from bayes_opt import BayesianOptimization
from sklearn.metrics import cohen_kappa_score

%matplotlib inline

%load_ext autoreload
%autoreload 2
from DSB_func_ishikawa import *

import warnings
warnings.filterwarnings('ignore')



In [2]:
path = './input/'
print('Loading...')
train = pd.read_csv(f'{path}train.csv')
test = pd.read_csv(f'{path}test.csv')
labels = pd.read_csv(f'{path}train_labels.csv')


Loading...


In [3]:
# train_df[(train_df['event_code']==4100) & (train_df['type'] == 'Assessment')]
dropping_in_data=[
    'installation_id','game_session', 'event_id', 'squeeze_target', \
    'type', 'event_code', 'event_data', 'timestamp', 'date', \
    'game_time', 'event_count', 'game_session|installation_id', 'world',\
    'title|installation_id']
#correctとincorrectを予測したほうがいいかも
dropping_in_labels=['game_session','installation_id','title','num_correct','num_incorrect', 'accuracy', 'accuracy_group']
print(train.dtypes)
print(dropping_in_data)
print(dropping_in_labels)


event_id           object
game_session       object
timestamp          object
event_data         object
installation_id    object
event_count         int64
event_code          int64
game_time           int64
title              object
type               object
world              object
dtype: object
['installation_id', 'game_session', 'event_id', 'squeeze_target', 'type', 'event_code', 'event_data', 'timestamp', 'date', 'game_time', 'event_count', 'game_session|installation_id', 'world', 'title|installation_id']
['game_session', 'installation_id', 'title', 'num_correct', 'num_incorrect', 'accuracy', 'accuracy_group']


In [4]:
new_train, new_test = transform(train, test)
if train['event_id'].all() == new_train['event_id'].all():
    print('transforming is done!')
new_train.sample(10)

game_session_feature:
100%|██████████| 12629/12629 [01:01<00:00, 206.08it/s]


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,...,exp_assess,(A)title->session_speed.mean,title|installation_id,(B)title|installation_id->session_speed.mean,(A)-(B),installation_id->hour.mean,installation_id->hour.var,installation_id->final_game_time.mean,installation_id->final_game_time.var,installation_id|transaction_count
614775,a8a78786,3aa3b96f510a327c,2019-09-24 13:51:34.196000+00:00,"{""description"":""You've made Epidermis so happy...",0d7a9eb6,51,3121,74893,Happy Camel,Game,...,0,2304.083836,Happy Camel|0d7a9eb6,1540.061333,7.640225e+02,10,53.282647,225017,2.276365e+10,585
3406404,76babcde,64cce62a5b5ba748,2019-10-07 16:03:52.201000+00:00,"{""coordinates"":{""x"":516,""y"":148,""stage_width"":...",4bcc7967,31,4070,48388,Dino Dive,Game,...,0,2880.631725,Dino Dive|4bcc7967,1453.545918,1.427086e+03,18,0.517196,149101,1.136653e+10,3328
5781066,c0415e5c,bb612dce30692e22,2019-09-08 23:25:56.752000+00:00,"{""size"":1,""dinosaur"":""mikey"",""water_level"":7,""...",80113e7b,94,4020,143381,Dino Dive,Game,...,0,2880.631725,Dino Dive|80113e7b,1491.570922,1.389061e+03,21,14.948286,447302,5.551209e+11,5657
592821,76babcde,3cb50cc7b00fff32,2019-09-11 22:15:25.493000+00:00,"{""coordinates"":{""x"":495,""y"":128,""stage_width"":...",0d5a1a5e,185,4070,203990,Dino Dive,Game,...,0,2880.631725,Dino Dive|0d5a1a5e,1693.687885,1.186944e+03,17,29.298207,492815,1.075295e+12,578
10619249,857f21c0,e2cb23359fdcd046,2019-09-10 23:38:29.369000+00:00,"{""current_containers"":8,""total_containers"":8,""...",f035b073,311,4040,253952,Bubble Bath,Game,...,0,3162.331866,Bubble Bath|f035b073,1065.501044,2.096831e+03,16,46.160822,242627,2.323178e+10,10638
4591271,ac92046e,87d055f054670239,2019-08-06 13:12:02.938000+00:00,"{""description"":""That's a pretty big hog there....",662e19b4,338,3120,311097,Scrub-A-Dub,Game,...,0,1794.569928,Scrub-A-Dub|662e19b4,848.644599,9.459253e+02,8,17.375818,217288,1.667546e+10,4523
11121950,cfbd47c8,15984d66f2f8bc16,2019-09-15 17:25:52.124000+00:00,"{""weight"":3,""source"":""scale"",""resources"":[1,3]...",fa3d2498,15,4030,29659,Chow Time,Game,...,0,2527.432975,Chow Time|fa3d2498,1382.045226,1.145388e+03,0,7.384372,212534,2.064188e+10,11048
5960872,7da34a02,9739a2c65c48413e,2019-09-23 19:16:02.861000+00:00,"{""coordinates"":{""x"":557,""y"":577,""stage_width"":...",84574d49,16,4070,0,Mushroom Sorter,Assessment,...,0,2025.015745,Mushroom Sorter|84574d49,2025.015745,8.678171e-09,19,2.219835,1065197,1.872177e+12,5831
4382315,2230fab4,149cee6a7e510fc0,2019-10-08 14:08:50.624000+00:00,"{""description"":""That's not enough food. Try pu...",60c5fd08,57,3120,88464,Chow Time,Game,...,0,2527.432975,Chow Time|60c5fd08,1220.752941,1.306680e+03,14,0.000000,103764,0.000000e+00,4289
146048,e57dd7af,59f33e30d9584f69,2019-10-08 18:07:11.946000+00:00,"{""description"":""Our team is too light, it need...",03560e11,87,3120,138800,Leaf Leader,Game,...,0,2741.303990,Leaf Leader|03560e11,1388.063380,1.353241e+03,16,3.654646,163991,1.165404e+10,153


In [5]:
sq_new_train = data_squeeze(new_train,mode='train')


data_squeeze:
100%|██████████| 21239/21239 [01:11<00:00, 296.54it/s]


In [6]:
labels = pd.read_csv('./input/train_labels.csv')
labels.loc[labels['num_incorrect'] > 8, 'num_incorrect'] = 8
labels['game_session|installation_id'] = labels['game_session'] +'|'+ labels['installation_id']
labels['(num_incorrect+1)*num_correct'] = (labels['num_incorrect']+ 1) * labels['num_correct']
dropped_labels = labels.drop(columns=dropping_in_labels, axis=1)

sq_new_train['game_session|installation_id'].head()
merged_sq_new_train = pd.merge(sq_new_train, dropped_labels, on='game_session|installation_id')
dropped_sq_new_train = merged_sq_new_train.drop(columns=dropping_in_data, axis=1)


In [7]:
#Bをその時点でのスピードにできればよい
#dropped_sq_new_train[dropped_sq_new_train['(A)-(B)'] <= -100].sample(10)


In [8]:
new_test.head()


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,...,exp_assess,(A)title->session_speed.mean,title|installation_id,(B)title|installation_id->session_speed.mean,(A)-(B),installation_id->hour.mean,installation_id->hour.var,installation_id->final_game_time.mean,installation_id->final_game_time.var,installation_id|transaction_count
4,7d093bf9,a022c3f60ba547e7,2019-09-10 16:54:12.115000+00:00,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,...,0,2527.432975,Chow Time|00abaee7,854.050314,1673.38266,15,2.60401,538280,5.065688e+11,35
5,f93fc684,a022c3f60ba547e7,2019-09-10 16:54:14.338000+00:00,"{""coordinates"":{""x"":515,""y"":697,""stage_width"":...",00abaee7,2,4010,2232,Chow Time,Game,...,0,2527.432975,Chow Time|00abaee7,854.050314,1673.38266,15,2.60401,538280,5.065688e+11,35
6,7ec0c298,a022c3f60ba547e7,2019-09-10 16:54:16.553000+00:00,"{""description"":""It's Chow Time! We have some V...",00abaee7,3,3010,4445,Chow Time,Game,...,0,2527.432975,Chow Time|00abaee7,854.050314,1673.38266,15,2.60401,538280,5.065688e+11,35
7,0d1da71f,a022c3f60ba547e7,2019-09-10 16:54:23.364000+00:00,"{""description"":""It's Chow Time! We have some V...",00abaee7,4,3110,11251,Chow Time,Game,...,0,2527.432975,Chow Time|00abaee7,854.050314,1673.38266,15,2.60401,538280,5.065688e+11,35
8,63f13dd7,a022c3f60ba547e7,2019-09-10 16:54:23.365000+00:00,"{""dinosaur"":""stacey"",""diet"":""herbivore"",""targe...",00abaee7,5,2020,11251,Chow Time,Game,...,0,2527.432975,Chow Time|00abaee7,854.050314,1673.38266,15,2.60401,538280,5.065688e+11,35


In [9]:
sq_new_test = data_squeeze(new_test, mode='test')
dropped_sq_new_test = sq_new_test.drop(columns=dropping_in_data, axis=1)


0%|          | 0/1000 [00:00<?, ?it/s]data_squeeze:
test_squeeze:
100%|██████████| 1000/1000 [00:02<00:00, 488.65it/s]


In [10]:
train = dropped_sq_new_train
test = dropped_sq_new_test


In [11]:
print(train.columns)
print(test.columns)


Index(['title', 'hour', 'weekday', 'exp_assess',
       '(A)title->session_speed.mean',
       '(B)title|installation_id->session_speed.mean', '(A)-(B)',
       'installation_id->hour.mean', 'installation_id->hour.var',
       'installation_id->final_game_time.mean',
       'installation_id->final_game_time.var',
       'installation_id|transaction_count', '(num_incorrect+1)*num_correct'],
      dtype='object')
Index(['title', 'hour', 'weekday', 'exp_assess',
       '(A)title->session_speed.mean',
       '(B)title|installation_id->session_speed.mean', '(A)-(B)',
       'installation_id->hour.mean', 'installation_id->hour.var',
       'installation_id->final_game_time.mean',
       'installation_id->final_game_time.var',
       'installation_id|transaction_count'],
      dtype='object')


In [12]:
train.head()

,title,hour,weekday,exp_assess,(A)title->session_speed.mean,(B)title|installation_id->session_speed.mean,(A)-(B),installation_id->hour.mean,installation_id->hour.var,installation_id->final_game_time.mean,installation_id->final_game_time.var,installation_id|transaction_count,(num_incorrect+1)*num_correct
0,4,5,1,0,2025.015745,2025.015745,8.678171e-09,11,43.285666,179263,1.718751e+10,2,1
1,0,5,1,0,2025.015745,2025.015745,7.764811e-09,11,43.285666,179263,1.718751e+10,2,0
2,4,5,1,1,2025.015745,2025.015745,8.678171e-09,11,43.285666,179263,1.718751e+10,2,1
3,4,20,1,3,2025.015745,2025.015745,8.678171e-09,11,43.285666,179263,1.718751e+10,2,2
4,0,20,1,1,2025.015745,2025.015745,7.764811e-09,11,43.285666,179263,1.718751e+10,2,1


In [13]:
categoricals = ['title', 'hour', 'weekday', 'exp_assess', 'installation_id->hour.mean']
def bo_run_lgb(n_estimators, subsample, learning_rate, feature_fraction, train=train, test=test):
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    target = train['(num_incorrect+1)*num_correct']
    train = train.drop(['(num_incorrect+1)*num_correct'], axis=1)
    oof_pred = np.zeros((len(train)))
    y_pred = np.zeros((len(test)))
    for fold, (tr_ind, val_ind) in enumerate(kf.split(train, target)):
        #print('Fold {}'.format(fold + 1))
        x_train, x_val = train.loc[tr_ind], train.loc[val_ind]
        y_train, y_val = target.loc[tr_ind], target.loc[val_ind]
        train_set = lgb.Dataset(x_train, y_train, categorical_feature=categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature=categoricals)

        params = {
            'n_estimators':int(n_estimators),
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'subsample': subsample,
            'subsample_freq': 1,
            'learning_rate': learning_rate,
            'feature_fraction': feature_fraction,
            'lambda_l1': 1,
            'lambda_l2': 1,
            'early_stopping_rounds': 100,
            'verbose': -1
            }

        model = lgb.train(params, train_set, early_stopping_rounds = 50, valid_sets=[train_set, val_set], verbose_eval=False)
        oof_pred[val_ind] = model.predict(x_val)
        #print('Partial score of fold {} is: {}'.format(fold, eval_qwk_lgb_regr(y_val, oof_pred[val_ind])[1]))
        y_pred += model.predict(test) / 5
    loss_score = cohen_kappa_score(target, np.round(oof_pred), weights='quadratic')#trainのロス
    result = pd.Series(np.argmax(oof_pred))
    #print('Our oof cohen kappa score is: ', loss_score)
    #print(result.value_counts(normalize = True))
    return -(loss_score)#model, y_pred


In [14]:
pbounds = {
    'n_estimators':(5000, 50000),
    'subsample': (0.5, 0.9),
    'learning_rate': (0.001, 0.01),
    'feature_fraction': (0.5, 0.9),
    }
optimizer = BayesianOptimization(f=bo_run_lgb, pbounds=pbounds)
optimizer.maximize(init_points=10, n_iter=30)

|   iter    |  target   | featur... | learni... | n_esti... | subsample |
-------------------------------------------------------------------------
|  1        | -0.1488   |  0.7818   |  0.008959 |  3.47e+04 |  0.7758   |
|  2        | -0.1472   |  0.5916   |  0.001423 |  8.016e+0 |  0.6222   |
|  3        | -0.15     |  0.7592   |  0.008439 |  2.339e+0 |  0.8001   |
|  4        | -0.1454   |  0.8839   |  0.00527  |  4.148e+0 |  0.5354   |
|  5        | -0.1523   |  0.6484   |  0.006356 |  1.028e+0 |  0.7175   |
|  6        | -0.1445   |  0.7988   |  0.001102 |  1.313e+0 |  0.8018   |
|  7        | -0.1438   |  0.8015   |  0.004667 |  2.983e+0 |  0.635    |
|  8        | -0.153    |  0.5354   |  0.006225 |  3.93e+04 |  0.519    |
|  9        | -0.1496   |  0.5137   |  0.003516 |  4.07e+04 |  0.6254   |
|  10       | -0.1407   |  0.8224   |  0.001669 |  1.792e+0 |  0.5301   |
|  11       | -0.1448   |  0.5623   |  0.002748 |  5e+04    |  0.5407   |
|  12       | -0.1545   |  0.5611   | 

In [20]:
def final_run_lgb(train=train, test=test, max_param=optimizer.max['params']):
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    target = train['(num_incorrect+1)*num_correct']
    train = train.drop(['(num_incorrect+1)*num_correct'], axis=1)
    oof_pred = np.zeros((len(train)))
    y_pred = np.zeros((len(test)))
    print(max_param)
    for fold, (tr_ind, val_ind) in enumerate(kf.split(train, target)):
        print('Fold {}'.format(fold + 1))
        x_train, x_val = train.loc[tr_ind], train.loc[val_ind]
        y_train, y_val = target.loc[tr_ind], target.loc[val_ind]
        train_set = lgb.Dataset(x_train, y_train, categorical_feature=categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature=categoricals)

        params = {
            'n_estimators':int(max_param['n_estimators']),
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'subsample': max_param['subsample'],
            'subsample_freq': 1,
            'learning_rate': max_param['learning_rate'],
            'feature_fraction': max_param['feature_fraction'],
            'lambda_l1': 1,
            'lambda_l2': 1,
            'early_stopping_rounds': 100,
            'verbose': -1
            }

        model = lgb.train(params, train_set, early_stopping_rounds = 50, valid_sets=[train_set, val_set], verbose_eval=1000)
        oof_pred[val_ind] = model.predict(x_val)
        print('Partial score of fold {} is: {}'.format(fold, cohen_kappa_score(y_val, np.round(oof_pred[val_ind]))))
        y_pred += model.predict(test) / 5
    loss_score = cohen_kappa_score(target, np.round(oof_pred), weights='quadratic')#trainのロス
    result = pd.Series(np.argmax(oof_pred))
    print('Our oof cohen kappa score is: ', loss_score)
    print(result.value_counts(normalize = True))
    return model, y_pred

In [21]:
model, prediction = final_run_lgb(train, test)

{'feature_fraction': 0.8223804299190307, 'learning_rate': 0.0016694026145288804, 'n_estimators': 17919.446553231028, 'subsample': 0.5301332047387168}
Fold 1
[1000]	training's rmse: 1.47789	valid_1's rmse: 1.53149
[2000]	training's rmse: 1.42617	valid_1's rmse: 1.52279
[3000]	training's rmse: 1.38557	valid_1's rmse: 1.52038
Partial score of fold 0 is: 0.0715437111622872
Fold 2
[1000]	training's rmse: 1.47919	valid_1's rmse: 1.527
[2000]	training's rmse: 1.42703	valid_1's rmse: 1.5178
Partial score of fold 1 is: 0.0736300798349544
Fold 3
[1000]	training's rmse: 1.47532	valid_1's rmse: 1.53419
[2000]	training's rmse: 1.42182	valid_1's rmse: 1.52951
Partial score of fold 2 is: 0.07854643860295785
Fold 4
[1000]	training's rmse: 1.47716	valid_1's rmse: 1.53018
[2000]	training's rmse: 1.42409	valid_1's rmse: 1.52137
[3000]	training's rmse: 1.38346	valid_1's rmse: 1.51798
Partial score of fold 3 is: 0.0865137408381027
Fold 5
[1000]	training's rmse: 1.47702	valid_1's rmse: 1.53978
[2000]	traini

In [22]:
importance = pd.DataFrame(model.feature_importance(), index=test.columns, columns=['importance']).sort_values('importance', ascending=False)
display(importance)

,importance
installation_id->final_game_time.mean,22171
installation_id|transaction_count,22018
installation_id->hour.var,21130
installation_id->final_game_time.var,21044
(A)-(B),14786
hour,14360
installation_id->hour.mean,13376
(B)title|installation_id->session_speed.mean,11397
title,6680
weekday,6425


In [23]:
#クラスタリングでもいいかも
sub = pd.read_csv(f'{path}sample_submission.csv')
sub['(num_incorrect+1)*num_correct'] = np.round(prediction.astype(int))
sub['accuracy_group'] = convert(sub)
sub = sub.drop('(num_incorrect+1)*num_correct', axis=1)
sub.to_csv('submission.csv', index = False)
display(sub['accuracy_group'].value_counts())
print(sub.columns.values)
print('done!')



1    793
0    171
2     35
3      1
Name: (num_incorrect+1)*num_correct, dtype: int64

3    794
0    171
2     35
Name: accuracy_group, dtype: int64

3    794
0    171
2     35
Name: accuracy_group, dtype: int64

['installation_id' 'accuracy_group']
done!
